In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
image = cv2.imread(r'C:\Users\welcome\Desktop\New folder\Drishti-GS1_files\Drishti-GS1_files\Test\Images/drishtiGS_014.png')


In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(10,10))
import xlrd
wb = xlrd.open_workbook(r"C:\Users\welcome\Desktop\New folder\Drishti-GS1_files\Drishti-GS1_diagnosis.xlsx") 
sheet = wb.sheet_by_index(0) 
val = [sheet.col_values(1)[5:],sheet.col_values(8)[5:]]
val

In [ ]:
def load_image(path):
    return cv2.imread(path)
import sys
import os
import glob
#function to load image and their name
def load_set(folder, shuffle=False):
    img_list = sorted(glob.glob(os.path.join(folder, '*.png')) + \
                      glob.glob(os.path.join(folder, '*.jpg')) + \
                      glob.glob(os.path.join(folder, '*.jpeg')))
    if shuffle:
        np.random.shuffle(img_list)
    data = []
    filenames = []
    for img_fn in img_list:
        img = load_image(img_fn)
        data.append(img)
        filenames.append(img_fn)
    return data, filenames
#DATA EXTRACTION FUNCTION
def extract_DRISHTI_GS_train(db_folder,cdr,train_data):
    db_folder=r'C:\Users\welcome\Desktop\New folder'
    file_codes_all,exp1,exp2,exp3,exp4 = [], [], [], [], []
    if train_data:
        set_path = os.path.join(db_folder, 'Drishti-GS1_files','Drishti-GS1_files', 'Training')
    else:
        set_path = os.path.join(db_folder, 'Drishti-GS1_files','Drishti-GS1_files', 'Test')
    images_path = os.path.join(set_path, 'Images')
    X_all, file_names = load_set(images_path)
    rel_file_names = [os.path.split(fn)[-1] for fn in file_names]
    rel_file_names_wo_ext = [fn[:fn.rfind('.')] for fn in rel_file_names]
    if train_data:
        file_codes = [fn[fn.find('_'):] for fn in rel_file_names_wo_ext]
    else:
        file_codes = [fn[fn.find('_'):] for fn in rel_file_names_wo_ext]
    file_codes_all.extend(file_codes)
    
    for fn in rel_file_names_wo_ext:
        if cdr:
            if train_data:
                CDR = open(os.path.join(set_path, 'GT', fn,fn + '_cdrValues.txt'),'r')
            else:
                CDR = open(os.path.join(set_path, 'Test_GT', fn,fn + '_cdrValues.txt'),'r')
            CDR = list(CDR)
            CDR = CDR[0].split()
            exp1.append(float(CDR[0]))
            exp2.append(float(CDR[1]))
            exp3.append(float(CDR[2]))
            exp4.append(float(CDR[3]))
            
    return X_all, file_codes_all,exp1,exp2,exp3,exp4,file_names

# GET DATA
X_all,file_codes_all,exp1,exp2,exp3,exp4,file_names = extract_DRISHTI_GS_train('',True,False) #put the folder where dataset is

print(np.array(X_all).shape)
print(np.array(X_all[0]).shape)

file_names

In [ ]:
# FUNCTION TO SEGMENT CUP AND DISK
def segment(image,plot_seg,plot_hist,name):

    image = image[400:1500,400:1500,:] 

    Abo,Ago,Aro = cv2.split(image) 
    M = 100
    filter = signal.gaussian(M, std=6)
    STDf = filter.std()
    

    Ar = Aro - Aro.mean() - Aro.std() 
    Mr = Ar.mean()          
    SDr = Ar.std()                          
    Thr = 0.5*M - STDf - SDr            

    
    M = 30
    filter_cup = signal.gaussian(M, std=6) 
    STDf = filter_cup.std()  

    Ag = Ago - Ago.mean() - Ago.std()
    Mg = Ag.mean()                        
    SDg = Ag.std()                        
    Thg = 0.5*M +2*STDf + 2*SDg + Mg     
    #print(Thg)
    
    
    hist,bins = np.histogram(Ag.ravel(),256,[0,256])  
    histr,binsr = np.histogram(Ar.ravel(),256,[0,256])


    smooth_hist_g=np.convolve(filter,hist)  
    smooth_hist_r=np.convolve(filter_cup,histr) 
    
    if plot_hist:
        plt.figure(figsize = (8,8))
        plt.subplot(2, 2, 1)
        plt.plot(hist)
        plt.title("Preprocessed Green Channel")

        plt.subplot(2, 2, 2)
        plt.plot(smooth_hist_g)
        plt.title("Smoothed Histogram Green Channel")

        plt.subplot(2, 2, 3)
        plt.plot(histr)
        plt.title("Preprocessed Red Channel")

        plt.subplot(2, 2, 4)
        plt.plot(smooth_hist_r)
        plt.title("Smoothed Histogram Red Channel")

        plt.show()
    
    r,c = Ag.shape
    Dd = np.zeros(shape=(r,c)) 
    Dc = np.zeros(shape=(r,c)) 

    for i in range(1,r):
        for j in range(1,c):
            if Ar[i,j]>Thr:
                Dd[i,j]=255
            else:
                Dd[i,j]=0

    for i in range(1,r):
        for j in range(1,c):
        
            if Ag[i,j]>Thg:
                Dc[i,j]=255
            else:
                Dc[i,j]=0
    
    name = name[47:]
    count=0
   
    cv2.imwrite('C:/Users/lenovo/Desktop/paper 1 results/disk.png',Dd)
        #count+=1
    plt.imsave('C:/Users/lenovo/Desktop/paper 1 results/cup.png',Dc)
    count+=1
    if plot_seg:
        plt.imshow(Dd, cmap = 'gray', interpolation = 'bicubic')
        plt.axis("off")
        plt.title("Optic Disk")
        plt.show()
        
        plt.imshow(Dc, cmap = 'gray', interpolation = 'bicubic')
        plt.axis("off")
        plt.title("Optic Cup")
        plt.show()

In [ ]:
# FUNCTION TO CALCULATE CDR
def cdr(cup,disc,plot):
    
    #morphological closing and opening operations
    R1 = cv2.morphologyEx(cup, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 5)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 2)
    img = R3
    ret,thresh = cv2.threshold(img,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) #Getting all possible contours in the segmented image
    cup_diameter = 0
    largest_area = 0
    el_cup = contours[0]
    if len(contours) != 0:
        for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv2.contourArea(contours[i]) 
                if (area>largest_area):
                    largest_area=area
                    index = i
                    el_cup = cv2.fitEllipse(contours[i])
                
    cv2.ellipse(img,el_cup,(140,60,150),3)  
    x,y,w,h = cv2.boundingRect(contours[index]) 
    cup_diameter = max(w,h) 

    #morphological closing and opening operations
    R1 = cv2.morphologyEx(disc, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 5)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)
    img2 = r3
    
    ret,thresh = cv2.threshold(img2,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    disk_diameter = 0
    largest_area = 0
    el_disc = el_cup
    if len(contours) != 0:
          for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv2.contourArea(contours[i]) 
                if (area>largest_area):
                    largest_area=area
                    index = i
                    el_disc = cv2.fitEllipse(contours[i])
                    
            cv2.ellipse(img2,el_disc,(140,60,150),10) 
            x,y,w,h = cv2.boundingRect(contours[index]) 
    disk_diameter = max(w,h)
                
    if plot:
        plt.imshow(img2, 'gray',interpolation = 'bicubic')
        plt.axis("off")
        plt.title("Optic Disk")
        plt.show()
        plt.imshow(img, 'gray')
        plt.axis("off")
        plt.title("Optic Cup")
        plt.show()
        
    if(disk_diameter == 0): return 1 
    cdr = cup_diameter/disk_diameter 
    return cdr

In [ ]:
from scipy import signal
table = []

# MAIN FUNCTION
CDR = [] 
VAL = [] 
count = 0

for i in range(len(X_all)):
    set_path = os.path.join(r'C:\Users\welcome\Desktop\New folder','Drishti-GS1_files','Drishti-GS1_files','Test',file_names[i]) 
    image = cv2.imread(set_path,1)
    segment(image,False,False,set_path)
    name = file_names[i][47:]
    cup = cv2.imread(r'C:\Users\welcome\Desktop\New folder\paper1results\cup.png',0) 
    disc = cv2.imread(r'C:\Users\welcome\Desktop\New folder\paper1results\disk.png',0) 
    #count+=1
    cdr_cal = cdr(cup,disc,False)
    if(val[1][int(file_codes_all[count][1:])-1] == 'Glaucomatous'):
        VAL.append(1)
    else:
        VAL.append(0)
    CDR.append(cdr_cal)
    if(cdr_cal <= 0.3):
        stri="Not glaucoma"
    elif(cdr_cal > 0.3 and cdr_cal <=0.5):
        stri="moderate glaucoma"
    elif(cdr_cal>0.5):
        stri="heavily glaucoma"
    else:
        stri=""
    table.append([file_codes_all[count], exp1[count], exp2[count], exp3[count], exp4[count], cdr_cal, stri])
   # table.append([file_codes_all[count], exp1[count], exp2[count], exp3[count], exp4[count], cdr_cal])
    count+=1
image = cv2.imread(set_path)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
table = pd.DataFrame(table, columns = ['File Code', 'Expert1', 'Expert2', 'Expert3', 'Expert4', 'predicted cdr'])
display(table)
table.to_csv('C:/Users/welcome/Desktop/New folder/output/result1.csv',index=True)